## import the required libraries 

In [1]:
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import seaborn
import pandas as pd
import numpy, scipy, matplotlib.pyplot as plt, sklearn, urllib, IPython.display
import librosa, librosa.display
plt.rcParams['figure.figsize'] = (14,5)

## Sound processing (create a list of gun_shots and non_gun_shots)

In [2]:
gun_shot_clip =[]
for i in range(1,1891):
    temp = str(i).zfill(6) 
    fileName = temp + ".wav"
    audio_path = "discharge-detection/training-data/train_"+fileName
    gun_shot_clip.append(audio_path)
    
other_clip = []
for i in range(1891,8523):
    temp = str(i).zfill(6) 
    fileName = temp + ".wav"
    audio_path = "discharge-detection/training-data/train_"+fileName
    other_clip.append(audio_path)
    

print "Total training data ", len(gun_shot_clip) + len(other_clip)

Total training data  8522


## Feature extraction using librosa

In [3]:
gunshot_signals = [
    librosa.load(p)[0] for p in gun_shot_clip
]
other_signals = [
    librosa.load(p)[0] for p in other_clip
]

In [4]:
def extract_features(signal):
    return [
        librosa.feature.zero_crossing_rate(signal)[0, 0],
        librosa.feature.spectral_centroid(signal)[0, 0],
    ]

## Create an array of gunshot_signals and other_signals

In [5]:
other_features = numpy.array([extract_features(x) for x in other_signals])
gunshot_features = numpy.array([extract_features(x) for x in gunshot_signals])

In [6]:
feature_table = numpy.vstack((gunshot_features, other_features))
print feature_table.shape

(8522, 2)


In [7]:
scaler = sklearn.preprocessing.MinMaxScaler(feature_range=(-1, 1))
training_features = scaler.fit_transform(feature_table)
print training_features.min(axis=0)
print training_features.max(axis=0)

[-1. -1.]
[ 1.  1.]


## Associate each feature vector with a class label

In [8]:
import numpy as np
training_labels = np.concatenate([np.zeros(1890), np.ones(6632)])

## Train the model (KNN classifier)

In [9]:
import pandas
from sklearn import model_selection
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    training_features, training_labels, test_size=0.3, random_state=0)

knn = KNeighborsClassifier()

## Calculate 10-fold cross-validation accuracy

In [10]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report
from sklearn import metrics

CV = 10

scores = cross_val_score(knn, X_train, y_train, cv=CV)
print("Training Accuracy : %0.2f " % (scores.mean()))

scores = cross_val_predict(knn, X_test, y_test, cv=CV)
print("Test Accuracy : %0.2f" %metrics.accuracy_score(y_test, scores))

from sklearn.cross_validation import cross_val_predict

y_pred = cross_val_predict(knn, training_features, training_labels, cv=CV)
print "Classification Report for 10-fold cross validation"
print(classification_report(training_labels, y_pred))

Training Accuracy : 0.95 
Test Accuracy : 0.94


/Users/tusharpandit/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Classification Report for 10-fold cross validation
             precision    recall  f1-score   support

        0.0       0.88      0.89      0.88      1890
        1.0       0.97      0.96      0.97      6632

avg / total       0.95      0.95      0.95      8522



## Fit the model on training data and make predictions on test data

In [11]:
knn.fit(X_train, y_train)
predictions = knn.predict(X_test)

## Performance metrics

In [12]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

print "\nAccuracy"
print(accuracy_score(y_test, predictions))

print "\nConfusion matrix"
print(confusion_matrix(y_test, predictions))

print "\nClassification Report"
print(classification_report(y_test, predictions))


Accuracy
0.948377004302

Confusion matrix
[[ 490   66]
 [  66 1935]]

Classification Report
             precision    recall  f1-score   support

        0.0       0.88      0.88      0.88       556
        1.0       0.97      0.97      0.97      2001

avg / total       0.95      0.95      0.95      2557



## Feature extraction for unseen data

In [13]:
test_Data =[]
for i in range(1,1211):
    temp = str(i).zfill(6) 
    fileName = temp + ".wav"
    audio_path = "discharge-detection/test-data/test_"+fileName
    test_Data.append(audio_path)
   
test_signals = [
    librosa.load(p)[0] for p in test_Data
]

test_features = numpy.array([extract_features(x) for x in test_signals])


## Run the model on testing data

In [14]:
lst=[]
i = 0
while i < len(test_features):
    predictionsOnTestData = knn.predict(test_features[i])
    temp = "test_"+str(i+1).zfill(6)
    lst.append([temp ,predictionsOnTestData[0]])
    i = i + 1


cols = ['FileName','IsGunshot']
dataFrame = pd.DataFrame(lst, columns=cols)

dataFrame.to_csv(r"discharge-detection/KNN_predictions.csv",index=False, sep=',')


/Users/tusharpandit/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/tusharpandit/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/tusharpandit/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either usi

In [15]:
print dataFrame 

         FileName  IsGunshot
0     test_000001        0.0
1     test_000002        1.0
2     test_000003        1.0
3     test_000004        1.0
4     test_000005        0.0
5     test_000006        1.0
6     test_000007        1.0
7     test_000008        1.0
8     test_000009        0.0
9     test_000010        0.0
10    test_000011        1.0
11    test_000012        0.0
12    test_000013        0.0
13    test_000014        1.0
14    test_000015        1.0
15    test_000016        1.0
16    test_000017        1.0
17    test_000018        1.0
18    test_000019        1.0
19    test_000020        1.0
20    test_000021        1.0
21    test_000022        0.0
22    test_000023        0.0
23    test_000024        0.0
24    test_000025        1.0
25    test_000026        1.0
26    test_000027        0.0
27    test_000028        1.0
28    test_000029        1.0
29    test_000030        0.0
...           ...        ...
1180  test_001181        1.0
1181  test_001182        1.0
1182  test_001